In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets.skyline12 import Skyline12
from sklearn.model_selection import train_test_split

skyline12 = Skyline12('/storage/skyline12/data')
train_set, test_set = train_test_split(list(skyline12), train_size=100, random_state=0)
train_set, validation_set = train_test_split(train_set, train_size=0.8, random_state=42)
len(train_set), len(validation_set), len(test_set)

(80, 20, 20)

In [3]:
import tensorflow as tf
from datasets.skyline12 import create_augment_fn
from functools import partial

NUM_CLASSES = 5
FOLDS = 20


def ds_gen(sample_set, folds):
    augment = create_augment_fn()

    def preprocess(x, y, z):
        x, y, z = augment(x, y, z)
        x = x.astype('float32')
        x /= 255.0
        y[y >= NUM_CLASSES - 1] = NUM_CLASSES - 1
        y = tf.keras.utils.to_categorical(y, num_classes=NUM_CLASSES)
        return x, y  # ignore z

    for _ in range(folds):
        for x, y, z in sample_set:
            yield preprocess(x, y, z)


train_ds = tf.data.Dataset.from_generator(
    partial(ds_gen, train_set, FOLDS),
    (tf.dtypes.float32, tf.dtypes.uint8),
    (tf.TensorShape([512, 512, 3]), tf.TensorShape([512, 512, NUM_CLASSES]))
)
validation_ds = tf.data.Dataset.from_generator(
    partial(ds_gen, validation_set, FOLDS // 2),
    (tf.dtypes.float32, tf.dtypes.uint8),
    (tf.TensorShape([512, 512, 3]), tf.TensorShape([512, 512, NUM_CLASSES]))
)

In [4]:
from models.unet import create_unet
from metrics import CategoricalMeanIou

unet = create_unet()
unet.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[CategoricalMeanIou(num_classes=5), 'accuracy'],
    run_eagerly=False
)

In [5]:
!mkdir -p temp
import wandb
from wandb.keras import WandbCallback
from callbacks import WandbLogPredictions

early_stopper = tf.keras.callbacks.EarlyStopping(patience=5)
checkpointer = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/baseline-weights.h5', save_best_only=True, save_weights_only=True)
class_labels = {
    0: 'Borders',
    1: 'Sky',
    2: 'Unknown',
    3: 'Unknown',
    4: 'Skyscrapers',
}
data_to_log = next(iter(validation_ds.batch(20)))
log_predictions = WandbLogPredictions(data_to_log, class_labels)
AUTOTUNE = tf.data.experimental.AUTOTUNE

wandb.init(project="skyline12-augmentations", tags=['baseline'])
unet.fit(
    train_ds.batch(3).cache('temp/').prefetch(AUTOTUNE),
    epochs=200,
    validation_data=validation_ds.batch(3).cache('temp/').prefetch(AUTOTUNE),
    callbacks=[early_stopper, WandbCallback(), log_predictions, checkpointer]
)

wandb: Wandb version 0.10.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/200
534/534 [==============================] - 660s 1s/step - loss: 0.8498 - categorical_mean_iou: 0.2502 - accuracy: 0.6821 - val_loss: 0.8302 - val_categorical_mean_iou: 0.2913 - val_accuracy: 0.7027
Epoch 2/200
534/534 [==============================] - 328s 613ms/step - loss: 0.6579 - categorical_mean_iou: 0.3293 - accuracy: 0.7638 - val_loss: 0.5928 - val_categorical_mean_iou: 0.3662 - val_accuracy: 0.7903
Epoch 3/200
534/534 [==============================] - 327s 613ms/step - loss: 0.5247 - categorical_mean_iou: 0.3910 - accuracy: 0.8160 - val_loss: 0.4129 - val_categorical_mean_iou: 0.4445 - val_accuracy: 0.8607
Epoch 4/200
534/534 [==============================] - 327s 613ms/step - loss: 0.4490 - categorical_mean_iou: 0.4236 - accuracy: 0.8459 - val_loss: 0.4036 - val_categorical_mean_iou: 0.4454 - val_accuracy: 0.8653
Epoch 5/200
534/534 [==============================] - 326s 611ms/step - loss: 0.4058 - categorical_mean_iou: 0.4434 - accuracy: 0.8608 - val_loss: 0.3